In [1]:
from train.pretrain import *

In [2]:
import pandas as pd
import plotly.express as px

def get_model_para_bar(model):
    data = []
    xmap ={}
    for name,p in model.named_parameters():
        key = name.split(".")[0]
        if key not in xmap:xmap[key]=len(xmap)
        data.append([name+f".{tuple(p.shape)}",np.prod(p.shape),key])  
    names_col = ['layer','Log10_ParaM', 'block']
    plot_df = pd.DataFrame(data=data,columns=names_col)
    fig = px.bar(plot_df, x='block', y='Log10_ParaM', color='layer' ,title='My plot',height=600)
    fig.update_layout(showlegend=False) 
    fig.show()

In [36]:
args=get_args(args=[])
#args.debug = True
local_rank = 0

args.train_set='2D7066N' 
args.use_time_stamp=0
args.history_length=1
args.time_step=2
args.model_type='AFNONet'
args.wrapper_model='Time_Projection_Model'
args.embed_dim=256*3
args.use_amp=0
args.model_depth=12
args.valid_batch_size=12
#args.n_heads=8
args.share_memory=False
args.input_channel= 928
args.output_channel= 70
args.unique_up_sample_channel = 70

In [37]:
from mltool.universal_model_util import get_model_para_detail

In [38]:
import numpy as np

In [39]:
args= parse_default_args(args)
print(args.model_kargs)

{'img_size': (32, 64), 'patch_size': (2, 2), 'in_chans': 928, 'out_chans': 70, 'fno_blocks': 4, 'embed_dim': 768, 'depth': 12, 'debug_mode': 0, 'double_skip': False, 'fno_bias': False, 'fno_softshrink': 0.0, 'history_length': 1, 'reduce_Field_coef': False, 'modes': (17, 33, 6), 'mode_select': 'normal', 'physics_num': 4, 'pred_len': 1, 'n_heads': 8, 'label_len': 3, 'canonical_fft': 1, 'unique_up_sample_channel': 70, 'share_memory': False}


In [40]:
model = eval(args.model_type)(**args.model_kargs)

In [41]:
get_model_para_bar(model)#2048 70

In [26]:
get_model_para_bar(model)#embed= 2048 210

In [30]:
get_model_para_bar(model)#768 210

In [35]:
get_model_para_bar(model)#768 30

In [91]:
args.gpu = args.local_rank = gpu  = local_rank
##### parse args: dataset_kargs / model_kargs / train_kargs  ###########
args= parse_default_args(args)
SAVE_PATH = get_ckpt_path(args)
args.SAVE_PATH = str(SAVE_PATH)
########## inital log ###################
logsys = create_logsys(args)


if args.distributed:
    if args.dist_url == "env://" and args.rank == -1:
        args.rank = int(os.environ["RANK"])
    if args.multiprocessing_distributed:
        # For multiprocessing distributed training, rank needs to be the
        # global rank among all the processes
        args.rank = args.rank * ngpus_per_node + local_rank
    logsys.info(f"start init_process_group,backend={args.dist_backend}, init_method={args.dist_url},world_size={args.world_size}, rank={args.rank}")
    dist.init_process_group(backend=args.dist_backend, init_method=args.dist_url,world_size=args.world_size, rank=args.rank)

model           = build_model(args)
#param_groups    = timm.optim.optim_factory.add_weight_decay(model, args.weight_decay)
optimizer,lr_scheduler,criterion = build_optimizer(args,model)
loss_scaler     = torch.cuda.amp.GradScaler(enabled=True)
logsys.info(f'use lr_scheduler:{lr_scheduler}')

args.pretrain_weight = args.pretrain_weight.strip()
logsys.info(f"loading weight from {args.pretrain_weight}")
start_epoch, start_step, min_loss = load_model(model.module if args.distributed else model, optimizer, lr_scheduler, loss_scaler, path=args.pretrain_weight, 
                    only_model= (args.mode=='fourcast') or (args.mode=='finetune' and not args.continue_train) ,loc = 'cuda:{}'.format(args.gpu))
if args.more_epoch_train:
    assert args.pretrain_weight
    print(f"detect more epoch training, we will do a copy processing for {args.pretrain_weight}")
    os.system(f'cp {args.pretrain_weight} {args.pretrain_weight}-epoch{start_epoch}')
logsys.info("done!")

log at checkpoints/ERA5_20/FEDformer.normal_select.M32-33-6_P1L3/history_6_time_step_7_pretrain-small_every_1_step/10_20_15_17_50-seed_42
mode                           ---> pretrain
train_set                      ---> small
batch_size                     ---> 4
valid_batch_size               ---> 12
epochs                         ---> 80
save_warm_up                   ---> 5
more_epoch_train               ---> 0
seed                           ---> 42
input_noise_std                ---> 0.0
do_final_fourcast              ---> 1
debug                          ---> 0
distributed                    ---> 0
rank                           ---> 0
continue_train                 ---> 0
model_type                     ---> FEDformer
patch_size                     ---> 8
img_size                       ---> (32, 64)
modes                          ---> 32,33,6
mode_select                    ---> normal
physics_num                    ---> 4
input_channel                  ---> 20
output_channel       

TypeError: Object of type LoggingSystem is not JSON serializable

In [7]:
train_dataset_tensor=None;
train_record_load=None;
valid_dataset_tensor=None;
valid_record_load=None

In [9]:
# =======================> start training <==========================
print(f"entering {args.mode} training in {next(model.parameters()).device}")
now_best_path = SAVE_PATH / 'backbone.best.pt'
latest_ckpt_p = SAVE_PATH / 'pretrain_latest.pt'


train_dataset, val_dataset, train_dataloader,val_dataloader = get_train_and_valid_dataset(args,
               train_dataset_tensor=train_dataset_tensor,train_record_load=train_record_load,
               valid_dataset_tensor=valid_dataset_tensor,valid_record_load=valid_record_load)
logsys.info(f"use dataset ==> {train_dataset.__class__.__name__}")
logsys.info(f"Start training for {args.epochs} epochs")
metric_list = ['loss']
master_bar        = logsys.create_master_bar(args.epochs)
master_bar.set_multiply_graph(figsize=(9,3),engine=[['plot','plot']],labels=[metric_list])
for epoch in master_bar:
    if epoch < start_epoch:continue
    if hasattr(model,'set_epoch'):model.set_epoch(epoch=epoch,epoch_total=args.epochs)
    if hasattr(model,'module') and hasattr(model.module,'set_epoch'):model.module.set_epoch(epoch=epoch,epoch_total=args.epochs)
    logsys.record('learning rate',optimizer.param_groups[0]['lr'],epoch)
    train_loss = run_one_epoch(epoch, start_step, model, criterion, train_dataloader, optimizer, loss_scaler,logsys,'train')
    if (not args.more_epoch_train) and (lr_scheduler is not None):lr_scheduler.step(epoch)
    #torch.cuda.empty_cache()
    #train_loss = single_step_evaluate(train_dataloader, model, criterion,epoch,logsys,status='train') if 'small' in args.train_set else -1
    val_loss   = run_one_epoch(epoch, start_step, model, criterion, val_dataloader, optimizer, loss_scaler,logsys,'valid')

    if (not args.distributed) or (args.rank == 0 and local_rank == 0) :
        logsys.info(f"Epoch {epoch} | Train loss: {train_loss:.6f}, Val loss: {val_loss:.6f}")
        logsys.record('train', train_loss, epoch)
        logsys.record('valid', val_loss, epoch)
        if use_wandb:wandb.log({"epoch":epoch,'train':train_loss,'valid':val_loss})
        if val_loss < min_loss:
            min_loss = val_loss
            if epoch > args.epochs//10:
                logsys.info(f"saving best model ....")
                save_model(model, path=now_best_path, only_model=True)
                logsys.info(f"done;")
            #if last_best_path is not None:os.system(f"rm {last_best_path}")
            #last_best_path= now_best_path
            logsys.info(f"The best accu is {val_loss}")
        logsys.record('best_loss', min_loss, epoch)
        update_experiment_info(experiment_hub_path,epoch,args)
        if epoch>args.save_warm_up:
            logsys.info(f"saving latest model ....")
            save_model(model, epoch+1, 0, optimizer, lr_scheduler, loss_scaler, min_loss, latest_ckpt_p)
            logsys.info(f"done ....")

if os.path.exists(now_best_path) and args.do_final_fourcast:
    logsys.info(f"we finish training, then start test on the best checkpoint {now_best_path}")
    start_epoch, start_step, min_loss = load_model(model.module if args.distributed else model, path=now_best_path, only_model=True)
    run_fourcast(args, model,logsys)
if use_wandb:wandb.finish()

entering pretrain training in cuda:0
in this dataset:ERA5CephSmallDataset, years/root args is disabled
in this dataset:ERA5CephSmallDataset, years/root args is disabled


2022-10-20 13:36:35,562 use dataset ==> ERA5CephSmallDataset
2022-10-20 13:36:35,564 Start training for 80 epochs


2022-10-20 13:37:24,661 Epoch 0 | Train loss: 0.313534, Val loss: 0.315764
2022-10-20 13:37:24,663 The best accu is 0.3157639503479004


2022-10-20 13:38:14,601 Epoch 1 | Train loss: 0.312141, Val loss: 0.314317
2022-10-20 13:38:14,604 The best accu is 0.31431692838668823


2022-10-20 13:39:00,876 Epoch 2 | Train loss: 0.269667, Val loss: 0.240780
2022-10-20 13:39:00,879 The best accu is 0.24077992141246796



KeyboardInterrupt



In [ ]:
if local_rank == 0:
    print(f"Start training for {args.epochs} epochs")

master_bar        = logsys.create_master_bar(args.epochs)
last_best_path = None
for epoch in master_bar:
    if epoch < start_epoch:continue
    train_one_epoch(epoch, start_step, model, criterion, train_dataloader, optimizer, loss_scaler,lr_scheduler, min_loss,logsys)
    lr_scheduler.step(epoch)
    #torch.cuda.empty_cache()
    train_loss = single_step_evaluate(train_dataloader, model, criterion,epoch,logsys)
    #train_loss = -1
    val_loss   = single_step_evaluate(val_dataloader, model, criterion,epoch,logsys)

    if rank == 0 and local_rank == 0:
        print(f"Epoch {epoch} | Train loss: {train_loss:.6f}, Val loss: {val_loss:.6f}")
        logsys.record('train', train_loss, epoch)
        logsys.record('valid', val_loss, epoch)
        if val_loss < min_loss:
            min_loss = val_loss
            print(f"saving best model ....")
            now_best_path = SAVE_PATH / f'backbone.best.pt'
            if epoch>args.save_warm_up:save_model(model, path=now_best_path, only_model=True)
            #if last_best_path is not None:os.system(f"rm {last_best_path}")
            #last_best_path= now_best_path
            print(f"done; the best accu is {val_loss}")
        logsys.record('best_loss', min_loss, epoch)
        

In [ ]:
if __name__ == '__main__':
    args = get_args()
    ngpus = ngpus_per_node = torch.cuda.device_count()
    args.world_size = -1
    args.dist_file  = None
    args.rank       = 0
    args.dist_backend = "nccl"
    args.multiprocessing_distributed = ngpus>1
    if not hasattr(args,'train_set'):args.train_set='large'
    ip = os.environ.get("MASTER_ADDR", "127.0.0.1")
    port = os.environ.get("MASTER_PORT", "54247")
    hosts = int(os.environ.get("WORLD_SIZE", "1"))  # number of nodes
    rank = int(os.environ.get("RANK", "0"))  # node id
    gpus = torch.cuda.device_count()  # gpus per node
    args.dist_url = f"tcp://{ip}:{port}"
    if args.world_size == -1 and "SLURM_NPROCS" in os.environ:
        args.world_size = int(os.environ["SLURM_NPROCS"])
        args.rank       = int(os.environ["SLURM_PROCID"])
        jobid           = os.environ["SLURM_JOBID"]

        hostfile        = "dist_url." + jobid  + ".txt"
        if args.dist_file is not None:
            args.dist_url = "file://{}.{}".format(os.path.realpath(args.dist_file), jobid)
        elif args.rank == 0:
            import socket
            ip = socket.gethostbyname(socket.gethostname())
            port = find_free_port()
            args.dist_url = "tcp://{}:{}".format(ip, port)
            #with open(hostfile, "w") as f:f.write(args.dist_url)
        else:
            import os
            import time
            while not os.path.exists(hostfile):
                time.sleep(1)
            with open(hostfile, "r") as f:
                args.dist_url = f.read()
        print("dist-url:{} at PROCID {} / {}".format(args.dist_url, args.rank, args.world_size))
    else:
        args.world_size = 1
    args.distributed = args.world_size > 1 or args.multiprocessing_distributed
    train_dataset_tensor=valid_dataset_tensor=None

    print("======== loading data ==========")
    if 'small' in args.train_set:
        if not args.fourcast:
            train_dataset_tensor = load_small_dataset_in_memory('train').share_memory_()
            valid_dataset_tensor = load_small_dataset_in_memory('valid').share_memory_()
        else:
            train_dataset_tensor = load_small_dataset_in_memory('test').share_memory_()
            valid_dataset_tensor = None
    else:
        if args.fourcast:
            train_dataset_tensor = load_test_dataset_in_memory(years=[2018],root="/nvme/zhangtianning/datasets/ERA5").share_memory_()
            valid_dataset_tensor = None
    print("=======done==========")
    print(train_dataset_tensor.shape)
    if args.multiprocessing_distributed:
        args.world_size = ngpus_per_node * args.world_size
        torch.multiprocessing.spawn(main_worker, nprocs=ngpus_per_node, args=(ngpus_per_node, args,train_dataset_tensor,valid_dataset_tensor))
    else:
        main_worker(0, ngpus_per_node, args,train_dataset_tensor,valid_dataset_tensor)